# Heat Detection in Dairy cattle using CNN

### Dividing that dataset into training and test set

In [1]:
import splitfolders
splitfolders.ratio('test/test', output="output1", ratio=(.8, 0.18, 0.02)) 

Copying files: 1326 files [00:21, 60.54 files/s] 


### Importing the libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Part-1 Data Preprocessing

### Preprocessing the Training set

In [3]:
# in order to make sure that all the input images are of same scale, we are using ImageDataGenerator (for data augmentation)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   vertical_flip = True,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('output1/train/',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 1060 images belonging to 2 classes.


### Preprocessing the test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('output1/val',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 237 images belonging to 2 classes.


## PART-2 Building the CNN

###  Initiating the CNN

In [5]:
cnn = tf.keras.models.Sequential()

#### Step-1 Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=128,
                               kernel_size=3,
                               activation='relu',
                               input_shape=[256, 256, 3]))

#### Step-2 Pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                  strides=2))

### Another Convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=256,
                               kernel_size=3,
                               activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                  strides=2))

#### Step-3 Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

#### Step-4 Full connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=256,
                              activation='relu'))

#### Step-5 Output layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=1,
                              activation='sigmoid'))

## Part-3 Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adadelta',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])
# if we had more than 2 outcomes then we had used 'category_cross entroy'.

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 80)

Epoch 1/80
34/34 [==============================] - 468s 14s/step - loss: 0.6829 - accuracy: 0.5415 - val_loss: 0.6815 - val_accuracy: 0.4557
Epoch 2/80
34/34 [==============================] - 466s 14s/step - loss: 0.6608 - accuracy: 0.6208 - val_loss: 0.6663 - val_accuracy: 0.5738
Epoch 3/80
34/34 [==============================] - ETA: 0s - loss: 0.6440 - accuracy: 0.6764 

In [ ]:
import os
import glob
testAtypical = os.path.join('output/test/Atypical', '*')
testAtypical = glob.glob(testAtypical)
testTypical = os.path.join('output/test/Typical', '*')
testTypical = glob.glob(testTypical)

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
for i in range(len(testAtypical)):
    path= testAtypical[i]
    test_image = image.load_img(path, target_size = (256, 256))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    training_set.class_indices
    if result[0][0] == 1:
      prediction = 'Typical'
    else:
      prediction = 'Atypical'
    print(prediction)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
for i in range(len(testTypical)):
    path= testTypical[i]
    test_image = image.load_img(path, target_size = (256, 256))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    training_set.class_indices
    if result[0][0] == 1:
      prediction = 'Typical'
    else:
      prediction = 'Atypical'
    print(prediction)

In [ ]:
cnn.save('adadelta.h5')

In [ ]:
def guess(img):
    import tensorflow as tf
    from tensorflow.keras.preprocessing import image
    import numpy as np
    cnn = tf.keras.models.load_model('salivaModel.h5')
    test_image = image.load_img(img, target_size = (256, 256))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    if result[0][0] == 1:
      prediction = 'Typical'
    else:
      prediction = 'Atypical'
    return prediction

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('sample.jpeg')
imgplot = plt.imshow(img)
result=guess('sample.jpeg')
print(f"image is {result}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xpoints = np.array([1, 10, 20, 30, 40, 50, 60, 70, 80])
y1points = np.flip(np.array([86.89, 86.51, 85.28, 84.81, 84.15, 82.74, 81.6, 80.47, 56.7]))
y2points = np.flip(np.array([80.59, 79.75, 76.79, 77.22, 78.48, 76.79, 75.11, 77.64, 54.43]))

plt.plot(xpoints, y1points, label = 'accuracy')
plt.plot(xpoints, y2points, label = 'val accuracy')
plt.legend(loc= 4)
plt.xlabel("no. of epochs ")
plt.ylabel("acurracy")
plt.title("accuracy vs val. accuracy comparision")
plt.show()